In [1]:
import os 
import numpy as np
import pandas as pd
import librosa
import pyworld
import time
import shutil
import matplotlib.pyplot as plt

from tools import *
from model import *

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
seed_value = 0
np.random.seed(seed_value)
torch.manual_seed(seed_value)

In [3]:
sampling_rate = 16000
num_mcep = 36
frame_period = 5.0
n_frames = 512
num_label = 6

In [4]:
model_dir = "../model/NTT_corevo/DPM"
model_name = "DPM_lr3_e10000_b4"

In [5]:
def model_load_DPM():
    model = PredictingModel()
    model.load_state_dict(torch.load(os.path.join(model_dir, model_name)))
    return model

In [6]:
def data_load(batch_size = 1, label = -1, path=""):
    data_list = []
    label_list = []
    
    if (label == -1):
        random_label = True 
    else:
        random_label =  False
        
    for i in range(batch_size):
        
        if random_label :
            label = np.random.randint(0, num_label)
        
        if path == "":
            sample_data_dir = os.path.join(data_dir, "labeled/{:02}".format(label))
            file = np.random.choice(os.listdir(sample_data_dir))
        else:
            sample_data_dir = "./"
            file = path
        
        frames = 0
        count = 0
        while frames < n_frames:

            wav, _ = librosa.load(os.path.join(sample_data_dir, file), sr = sampling_rate, mono = True)
            wav = librosa.util.normalize(wav, norm=np.inf, axis=None)
            wav = wav_padding(wav = wav, sr = sampling_rate, frame_period = frame_period, multiple = 4)
            f0, timeaxis, sp, ap, mc = world_decompose(wav = wav, fs = sampling_rate, frame_period = frame_period, num_mcep = num_mcep)

            if (count == 0):
                mc_transposed = np.array(mc).T
            else:
                mc_transposed = np.concatenate([mc_transposed, np.array(mc).T], axis =1)
            frames = np.shape(mc_transposed)[1]

            mean = np.mean(mc_transposed)
            std = np.std(mc_transposed)
            mc_norm = (mc_transposed - mean)/std

            count += 1

        start_ = np.random.randint(frames - n_frames + 1)
        end_ = start_ + n_frames

        data_list.append(mc_norm[:,start_:end_])
        label_list.append(label)

    return torch.Tensor(data_list).view(batch_size, 1, num_mcep, n_frames), torch.Tensor(label_list).view(batch_size, 1)


In [ ]:
submit_df = pd.DataFrame()
model = model_load_DPM()
model.eval()

PredictingModel(
  (conv1): Conv2d(1, 8, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
  (conv1_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_gated): Conv2d(1, 8, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
  (conv1_gated_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_sigmoid): Sigmoid()
  (conv2): Conv2d(8, 16, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
  (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_gated): Conv2d(8, 16, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
  (conv2_gated_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_sigmoid): Sigmoid()
  (conv3): Conv2d(16, 32, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
  (conv3_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3_gated): Conv2d(16, 32, kernel_size=(4, 8), st

In [ ]:
feature_data_dir = "../feature_data/NTT_corevo"
data_dir = "../data/NTT_corevo"
train_data_dir = os.path.join(data_dir, "train")
test_data_dir = os.path.join(data_dir, "test")

list_id = []
list_label = []


for i, file in enumerate(os.listdir(test_data_dir)):
    list_id.append(file[:-4])
    data, _ = data_load(path=os.path.join(test_data_dir, file))
    label = np.argmax(model.predict(data).data.numpy().reshape(-1))
    if (label == 0):
        list_label.append("FE_CH")
    elif (label == 1):
        list_label.append("FE_AD")
    elif (label == 2):
        list_label.append("FE_EL")
    elif (label == 3):
        list_label.append("MA_CH")
    elif (label == 4):
        list_label.append("MA_AD")
    elif (label == 5):
        list_label.append("MA_EL")
    if (i % 100 == 0):
        print("{:02}%".format(i*100/len(os.listdir(test_data_dir))))
        
submit_df["id"] = list_id
submit_df["label"] = list_label

0.0%
0.5590339892665475%
1.118067978533095%
1.6771019677996422%


In [ ]:
submit_df.tail()

In [ ]:
submit_df.to_csv(os.path.join(data_dir, "submit.tsv"), sep='\t', header=False, index=False)